In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
print("Started")
import numpy as np, pandas as pd
print("Pandas version - ", pd.__version__)
from pandas import MultiIndex, Int16Dtype

try:
    import seaborn as sns
except:
    !pip install seaborn
    import seaborn as sns
import matplotlib.pyplot as plt
import time
#import cupy, cudf
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Importing RFE and LinearRegression
try:
    import statsmodels.api as sm
except:
    !python -m pip install statsmodels
    import statsmodels.api as sm

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model,tree,svm

try:
    import pickle
except:
    !pip install pickle5
    import pickle

from sklearn.metrics import mean_absolute_error
print("All necessary libraries imported")


Started
Pandas version -  1.3.5
All necessary libraries imported


In [3]:
# Not going to use testdf and sample df for thesis
traindf = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
testdf = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
sample_sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

split= False

print(traindf.shape, " - Training dataset shape")
print(testdf.shape, " - Testing dataset shape")
print(sample_sub.shape, " - Sample Submission dataset shape")

(6036000, 8)  - Training dataset shape
(4024000, 7)  - Testing dataset shape
(4024000, 2)  - Sample Submission dataset shape


In [4]:
# Adding features in the dataframe

def add_features(df):
    #https://www.kaggle.com/code/papernist/google-brain
    df = df.copy()
    
    df_group = df.groupby(['breath_id'])
    
    
    feature_list = ['u_in', 'time_step', 'cross']
    
    df['cross']= df['u_in'] * df['u_out']
    df['area_out']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    
    # add
    df['air_flow_rate'] = df['u_out'] - (df['u_in']/100.0)
    df['air_flow_area'] = df['air_flow_rate'] * df['time_step']
    print("Step-1...Completed")
    
    # cumsum cummean
    df['one'] = 1
    df['count'] = df_group['one'].cumsum()
    for feature in feature_list:
        df[f'{feature}_cumsum'] = df_group[feature].cumsum()
        df[f'{feature}_cummean'] = df[f'{feature}_cumsum'] / df['count']
        
    print("Step-2 cumsum cummean ...Completed")
    # lagging
    use_lags = 4
    for lag in range(1, use_lags+1):
        for feature in feature_list:
            # lag 
            df[f'{feature}_lag_{lag}'] = df_group[feature].shift(lag)
            # inverse lag
            df[f'{feature}_lag_inverse_{lag}'] = df_group[feature].shift(-lag)

            # dif lag
            df[f'{feature}_lag_diff_{lag}'] = df[feature] - df[f'{feature}_lag_{lag}']

            # dif inverse lag
            df[f'{feature}_lag_inverse_diff_{lag}'] = df[feature] - df[f'{feature}_lag_inverse_{lag}']

            df = df.drop(columns=[f'{feature}_lag_{lag}', f'{feature}_lag_inverse_{lag}'])
        
    df = df.fillna(0)
    print("Step-3 lagging ...Completed")
    
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    df = df.fillna(0)
    print("Step-4 categorical ...Completed")
    
    rolling_list = [15]
    for roll in rolling_list:
        for feature in feature_list:
            df[[f'{roll}_{feature}_sum',f'{roll}_{feature}_min',
                f'{roll}_{feature}_max',f'{roll}_{feature}_mean']] = (df_group[feature]\
                                                                  .rolling(window=roll,min_periods=1)\
                                                                  .agg({f'{roll}_{feature}_sum':'sum',
                                                                        f'{roll}_{feature}_min':'min',
                                                                        f'{roll}_{feature}_max':'max',
                                                                        f'{roll}_{feature}_mean':'mean'})\
                                                                   .reset_index(level=0,drop=True))
    
    print("Step-5 Sliding window...Completed")
    print()
    
    df = df.fillna(0)
    df = df.drop(['id', 'breath_id','one','count'], axis=1)
    
    return df.astype(np.float16)

In [5]:
def cross_val(model):
    pred = cross_val_score(model, X_train, y_train, cv=10)
    return pred.mean()


def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square    

In [6]:
%%time
train = add_features(traindf)
#test = add_features(testdf)

print("*"*70)
print(train.shape, " - Feature added training dataset shape")
#print(test.shape, " - Features added testing dataset shape")

Step-1...Completed
Step-2 cumsum cummean ...Completed
Step-3 lagging ...Completed
Step-4 categorical ...Completed
Step-5 Sliding window...Completed

**********************************************************************
(6036000, 66)  - Feature added training dataset shape
CPU times: user 1min 53s, sys: 42.6 s, total: 2min 36s
Wall time: 2min 34s


In [7]:
binary_col = []
num_col = []

for columnname in train.columns:
  if len(set(train[columnname])) != 2:
    num_col.append(columnname)
  else:
    binary_col.append(columnname)

train_numcol = num_col.copy()
train_numcol.remove('pressure')

print(len(binary_col), "Number of Binary Categorical Columns")
print(len(num_col),"Number of Numerical Columns")
print(len(binary_col)+len(num_col), "Sum of Binary & Numerical Columns")

16 Number of Binary Categorical Columns
50 Number of Numerical Columns
66 Sum of Binary & Numerical Columns


In [8]:
def cross_val(model):
    pred = cross_val_score(model, X_train, y_train, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square    

In [9]:
df_train, df_test = train_test_split(train, train_size = 0.7, test_size = 0.3, random_state = 100)
print("Split Done")

from sklearn import metrics
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split

y_train = df_train['pressure']
X_train = df_train.drop('pressure',axis=1)
X_train_copy = X_train.copy()

y_test = df_test['pressure']
X_test = df_test.drop('pressure',axis=1)
X_test_copy = X_test.copy()

print("*"*70)
print("Splitting the data into X train & Y train")
print(X_train.shape, "X train shape")
print(y_train.shape, "Y train shape")

print("*"*70)
print("Splitting the data into X test & Y test")
print(X_test.shape, "X test shape")
print(y_test.shape, "Y test shape")

Split Done
**********************************************************************
Splitting the data into X train & Y train
(4225200, 65) X train shape
(4225200,) Y train shape
**********************************************************************
Splitting the data into X test & Y test
(1810800, 65) X test shape
(1810800,) Y test shape


In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', MinMaxScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

In [11]:
results_df = []

In [12]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=100, solver='cholesky', tol=0.0001, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

test_pred = model.predict(X_test)
train_pred = model.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('====================================')
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df_2 = pd.DataFrame(data=[["Ridge Regression", *evaluate(y_test, test_pred) , cross_val(Ridge())]], 
                            columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])
#results_df = results_df.append(results_df_2, ignore_index=True)

Test set evaluation:
_____________________________________
MAE: 2.701533854955271
MSE: 17.259531696098957
RMSE: 4.154459254355368
R2 Square 0.7371331024119776
__________________________________
Train set evaluation:
_____________________________________
MAE: 2.706844363112645
MSE: 17.347890208228772
RMSE: 4.165079856164677
R2 Square 0.7363716455769249
__________________________________


In [13]:
results_df_2

,Model,MAE,MSE,RMSE,R2 Square,Cross Validation
0,Ridge Regression,2.701534,17.259532,4.154459,0.737133,0.737305
